In [31]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine 
import configparser 
import datetime

from sdv.single_table import GaussianCopulaSynthesizer
from sdv.metadata import Metadata
import pickle
import sdv

In [36]:
print(sdv.__version__)


1.17.1


In [32]:
#Берем данные по БД

config = configparser.ConfigParser()
config.read('config.ini')
conn_string = config.get('DATABASE', 'connection_url')

In [ ]:
#Чтение из БД

In [33]:
table = 'house_prices_train'
query = f''' 
select * from {table}
'''
def reading_from_db(query,table):
    db = create_engine(conn_string)
    
    with db.connect() as conn:
        df = pd.read_sql(query, con=conn)
        
    return df


In [34]:
df = reading_from_db(query,table)

In [35]:
df.head(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,dt
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,...,None,None,None,0,2,2008,WD,Normal,208500,2024-10-15 11:09:15
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,...,None,None,None,0,5,2007,WD,Normal,181500,2024-10-15 11:09:15
2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,...,None,None,None,0,9,2008,WD,Normal,223500,2024-10-15 11:09:15
3,4,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,...,None,None,None,0,2,2006,WD,Abnorml,140000,2024-10-15 11:09:15
4,5,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,...,None,None,None,0,12,2008,WD,Normal,250000,2024-10-15 11:09:15


In [ ]:
#Генерация

In [36]:
metadata = Metadata.detect_from_dataframe(
    data=df,
    table_name='house_prices')

synthesizer = GaussianCopulaSynthesizer(metadata)
synthesizer.fit(df)

synthetic_data = synthesizer.sample(num_rows=10)

/home/aleksey/anaconda3/lib/python3.12/site-packages/sdv/single_table/base.py:105: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


In [37]:
synthetic_data

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,dt
0,982268879,185,RL,69.0,11584,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,515,4,2007,New,Partial,275890,2024-10-15 11:09:15
1,804069452,108,RL,101.0,8706,Pave,NaN,IR3,Lvl,AllPub,...,NaN,NaN,NaN,575,4,2009,WD,Partial,246788,2024-10-15 11:09:15
2,839063981,48,RL,NaN,11230,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,2064,8,2009,WD,Normal,225499,2024-10-15 11:09:15
3,82149587,124,RL,71.0,6560,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,8,9,2006,WD,Partial,164671,2024-10-15 11:09:15
4,204237279,93,RL,91.0,14465,Pave,NaN,Reg,Lvl,AllPub,...,NaN,MnPrv,NaN,7,8,2009,WD,Normal,83705,2024-10-15 11:09:15
5,975382569,184,RL,NaN,1595,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,3303,4,2007,WD,Normal,91521,2024-10-15 11:09:15
6,952095692,33,RL,NaN,11269,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,0,5,2006,WD,Normal,116355,2024-10-15 11:09:15
7,152067842,143,RL,44.0,8909,Pave,NaN,Reg,Lvl,AllPub,...,NaN,MnPrv,NaN,52,8,2009,WD,Normal,287552,2024-10-15 11:09:15
8,536692407,64,RL,54.0,8696,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,6185,7,2008,WD,Abnorml,244868,2024-10-15 11:09:15
9,86284612,98,RL,95.0,11287,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,3695,6,2008,WD,Normal,158684,2024-10-15 11:09:15


In [38]:
#Сохранение модели для генерации случайных данных 
'''
with open("my_synthesizer.pkl", "wb") as f:
    pickle.dump(synthesizer, f)
'''
synthesizer.save('synthesizer.pkl')

'''synthesizer = GaussianCopulaSynthesizer.load('my_synthesizer.pkl')'''

"synthesizer = GaussianCopulaSynthesizer.load('my_synthesizer.pkl')"

In [11]:
synthetic_data['dt'] = datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')
#Баг с обучением
synthetic_data['3SsnPorch'] = 0

In [39]:
synthesizer = GaussianCopulaSynthesizer.load('my_synthesizer.pkl')

In [52]:
synthesizer.sample(num_rows=10)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,dt
0,298983288,45,RL,35.0,1793,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,702,4,2007,WD,Partial,143281,2024-10-15 11:09:15
1,841937430,34,RL,59.0,6015,Pave,NaN,IR1,Lvl,AllPub,...,NaN,MnPrv,NaN,1177,10,2008,WD,Normal,87520,2024-10-15 11:09:15
2,196418606,99,RL,58.0,12093,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,4500,5,2006,WD,Normal,92466,2024-10-15 11:09:15
3,675011281,146,RL,NaN,10081,Pave,NaN,IR2,Lvl,AllPub,...,NaN,NaN,NaN,6368,2,2007,WD,Normal,175422,2024-10-15 11:09:15
4,760407711,71,RL,68.0,5340,Grvl,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,481,8,2010,WD,Partial,295116,2024-10-15 11:09:15
5,452051114,87,RL,41.0,9593,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,3714,10,2007,New,Normal,74476,2024-10-15 11:09:15
6,996365303,67,RL,83.0,19535,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,2585,2,2010,WD,Normal,166213,2024-10-15 11:09:15
7,525762750,34,RL,102.0,5483,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,1,8,2010,WD,Normal,210077,2024-10-15 11:09:15
8,302808450,48,RL,43.0,3140,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,209,1,2006,WD,Partial,102529,2024-10-15 11:09:15
9,351409244,94,RL,NaN,16128,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,10,3,2006,WD,Normal,308158,2024-10-15 11:09:15


In [ ]:
#Запись в БД

In [14]:
#Берем данные по БД

config = configparser.ConfigParser()
config.read('config.ini')
conn_string = config.get('DATABASE', 'connection_url')

In [10]:
#Функция чтения и записи в БД

def upload_to_db(df_in,db_table_out):
    try:

        db = create_engine(conn_string) 
        with db.connect() as conn:
            
            df_in.to_sql(db_table_out, con=conn, if_exists='append',index=False) 
            print('Uploaded')
    finally:
        print('OK')


In [13]:
df_in = synthetic_data
db_table_out = 'house_prices_generator'

upload_to_db(df_in,db_table_out)

Uploaded
OK


In [1]:
import pandas as pd
import numpy as np
import configparser
from sqlalchemy import create_engine  
import datetime
import pickle
from sdv.single_table import GaussianCopulaSynthesizer

config = configparser.ConfigParser()
config.read('/home/aleksey/Notebooks_Projects/House-Prices-Airflow-Superset/Training Models And Test/config.ini')
conn_string = config.get('DATABASE', 'connection_url')


#Таблица с новыми данными которые генерируются

new_data_from_generator = 'house_prices_generator'

filepath = '/home/aleksey/Notebooks_Projects/House-Prices-Airflow-Superset/Training Models And Test/my_synthesizer.pkl'

with open(filepath,'rb') as f:
    synthesizer = pickle.load(f)

def upload_generator_data(new_data_from_generator):



    #Генерация случайных данных
    
    synthetic_data = synthesizer.sample(num_rows=10)

    #Баг с обучением

    synthetic_data['3SsnPorch'] = 0

    #Убираем баг и подставляем дату

    synthetic_data['dt'] = datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')
    
    engine = create_engine(conn_string) 
    
    '''with engine.connect() as conn:
        synthetic_data.to_sql(new_data_from_generator, con=conn, if_exists='replace',index=False) 
        '''
    
    return synthetic_data

In [30]:
upload_generator_data(new_data_from_generator)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,dt
0,319101530,75,RL,64.0,9159,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,75,10,2009,WD,Normal,348765,2024-10-15 21:10:04
1,576516507,88,RL,NaN,29413,Pave,NaN,IR1,Low,AllPub,...,NaN,NaN,NaN,364,3,2010,WD,Normal,205348,2024-10-15 21:10:04
2,182152631,66,RL,80.0,7787,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,1100,5,2008,WD,Normal,303685,2024-10-15 21:10:04
3,635606118,30,RL,115.0,10194,Pave,NaN,Reg,Lvl,AllPub,...,NaN,GdWo,NaN,248,5,2009,WD,Normal,310827,2024-10-15 21:10:04
4,6589289,153,RM,62.0,3632,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,6168,9,2008,WD,Normal,240502,2024-10-15 21:10:04
5,404229546,38,RL,62.0,13538,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,87,2,2008,WD,Normal,159996,2024-10-15 21:10:04
6,447243370,132,C (all),NaN,15654,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,5364,2,2006,WD,Normal,187883,2024-10-15 21:10:04
7,731088699,98,RL,80.0,10253,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,136,6,2007,WD,Normal,240629,2024-10-15 21:10:04
8,220152686,164,RL,76.0,12897,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,105,5,2009,WD,Normal,417038,2024-10-15 21:10:04
9,41535268,49,RM,58.0,6092,Pave,NaN,Reg,Low,AllPub,...,NaN,NaN,NaN,290,11,2006,WD,Normal,73976,2024-10-15 21:10:04
